In [24]:
import pandas as pd

from urllib.parse import quote
import urllib
import requests
import magic

# from simple_image_download import simple_image_download as simp

# response = simp.simple_image_download()
# images = ["156045004", "0060502461", "1Z0959858"]

# for i in images:
#     print(response.urls(i, 1, extensions={'.jpg'}))


def _download_page(url):
    try:
        headers = {}
        headers['User-Agent'] = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"
        req = urllib.request.Request(url, headers=headers)
        resp = urllib.request.urlopen(req)
        resp_data = str(resp.read())
        return resp_data
    except Exception as e:
        print(e)


def google_urls(keywords, limit, extensions={'.jpg', '.png', '.ico', '.gif', '.jpeg'}):
    keyword_to_search = [str(item).strip() for item in keywords.split(',')]
    i = 0
    links = []

    while i < len(keyword_to_search):
        url = 'https://www.google.com/search?q=' + quote(
            keyword_to_search[i].encode(
                'utf-8')) + '&biw=1536&bih=674&tbm=isch&sxsrf=ACYBGNSXXpS6YmAKUiLKKBs6xWb4uUY5gA:1581168823770&source=lnms&sa=X&ved=0ahUKEwioj8jwiMLnAhW9AhAIHbXTBMMQ_AUI3QUoAQ'
        raw_html = _download_page(url)

        end_object = -1
        google_image_seen = False
        j = 0

        while j < limit:
            tries = 0
            while True:
                try:
                    new_line = raw_html.find('"https://', end_object + 1)
                    end_object = raw_html.find('"', new_line + 1)

                    buffor = raw_html.find('\\', new_line + 1, end_object)
                    if buffor != -1:
                        object_raw = (raw_html[new_line + 1:buffor])
                    else:
                        object_raw = (raw_html[new_line + 1:end_object])

                    if any(extension in object_raw for extension in extensions):
                        break

                    tries += 1

                    if tries == 500:
                        object_raw = None
                        break
                except Exception as e:
                    object_raw = None
                    break

            print(tries)

            if object_raw is not None:
                try:
                    r = requests.get(object_raw, allow_redirects=True, timeout=1)
                    if('html' not in str(r.content)):
                        mime = magic.Magic(mime=True)
                        file_type = mime.from_buffer(r.content)
                        file_extension = f'.{file_type.split("/")[1]}'
                        if file_extension == '.png' and not google_image_seen:
                            google_image_seen = True
                            raise ValueError()
                        links.append(object_raw)
                    else:
                        j -= 1
                except Exception as e:
                    j -= 1
            j += 1

        i += 1

    return(links)


In [16]:
data = "C:/Users/aznavouridis.k/Desktop/Terpos/GBG_15_replaced.xlsx"

In [21]:
df = pd.read_excel(data, dtype='string')

In [25]:
a = df['description'].str.split(': ', expand=True)
a.columns = ['text', 'sku']
a['url'] = ''

In [26]:
for i in a.itertuples():
    if i.sku != '':
        url = google_urls(str(i.sku), 1, extensions={'.jpg'})
        try:
            a.loc[i.Index, 'url'] = url[0]
        except IndexError:
            continue


43
93
95
115
145
35
500
82
91
85
81
45
41
85
83


In [28]:
df['image'] = a['url']

In [29]:
df

,brand,article_no,title,description,meta_title_seo,details,retail_price,price_after_discount,id_category,image,meta_seo,extra_description
0,Alfa Romeo,065606440,ΒΕΝΤΙΛΑΤΕΡ ΨΥΓΕΙΟΥ ΚΟΜΠ.Χ.Α/C Alfa Romeo 33 19...,Γνήσιος κωδικός: 60504296,test ΒΕΝΤΙΛΑΤΕΡ ΨΥΓΕΙΟΥ ΚΟΜΠ.Χ.Α/C Alfa Romeo ...,"Μάρκα αυτοκινήτου: Alfa Romeo, Μοντέλο: 33, Χρ...","71,13","96,03",1111,https://static.car.gr/10498770_0_z.jpg,test ΒΕΝΤΙΛΑΤΕΡ ΨΥΓΕΙΟΥ ΚΟΜΠ.Χ.Α/C Alfa Romeo ...,ALFA ROMEO ALFA 33 83-90
1,Alfa Romeo,065807042,ΓΡΥΛΛΟΣ ΠΑΡΑΘΥΡΟ ΕΜΠΡΟΣ ΗΛΕΚΤΡΙΚΟΣ (ΕΥΡΩΠΗ) ΑΡ...,Γνήσιος κωδικός: 60573687,test ΓΡΥΛΛΟΣ ΠΑΡΑΘΥΡΟ ΕΜΠΡΟΣ ΗΛΕΚΤΡΙΚΟΣ (ΕΥΡΩΠ...,"Μάρκα αυτοκινήτου: Alfa Romeo, Μοντέλο: 33, Χρ...","38,89","52,5",1111,https://alfa-restoration.co.uk/shop/images/pro...,test ΓΡΥΛΛΟΣ ΠΑΡΑΘΥΡΟ ΕΜΠΡΟΣ ΗΛΕΚΤΡΙΚΟΣ (ΕΥΡΩΠ...,"ALFA ROMEO ALFA 33 83-90, ALFA ROMEO ALFA 33 9..."
2,Alfa Romeo,065807041,ΓΡΥΛΛΟΣ ΠΑΡΑΘΥΡΟ ΕΜΠΡΟΣ ΗΛΕΚΤΡΙΚΟΣ (ΕΥΡΩΠΗ) ΔΕ...,Γνήσιος κωδικός: 60573516,test ΓΡΥΛΛΟΣ ΠΑΡΑΘΥΡΟ ΕΜΠΡΟΣ ΗΛΕΚΤΡΙΚΟΣ (ΕΥΡΩΠ...,"Μάρκα αυτοκινήτου: Alfa Romeo, Μοντέλο: 33, Χρ...","38,89","52,5",1111,https://e-erotokritos.gr/wp-content/uploads/20...,test ΓΡΥΛΛΟΣ ΠΑΡΑΘΥΡΟ ΕΜΠΡΟΣ ΗΛΕΚΤΡΙΚΟΣ (ΕΥΡΩΠ...,"ALFA ROMEO ALFA 33 83-90, ALFA ROMEO ALFA 33 9..."
3,Alfa Romeo,065605492,ΓΩΝΙΑ ΦΛΑΣ ΛΕΥΚΗ ΑΡ Alfa Romeo 33 1983-1990,Γνήσιος κωδικός:,test ΓΩΝΙΑ ΦΛΑΣ ΛΕΥΚΗ ΑΡ Alfa Romeo 33 1983-1990,"Μάρκα αυτοκινήτου: Alfa Romeo, Μοντέλο: 33, Χρ...","9,03","12,19",1111,,test ΓΩΝΙΑ ΦΛΑΣ ΛΕΥΚΗ ΑΡ Alfa Romeo 33 1983-1990,ALFA ROMEO ALFA 33 83-90
4,Alfa Romeo,065605491,ΓΩΝΙΑ ΦΛΑΣ ΛΕΥΚΗ ΔΕ Alfa Romeo 33 1983-1990,Γνήσιος κωδικός:,test ΓΩΝΙΑ ΦΛΑΣ ΛΕΥΚΗ ΔΕ Alfa Romeo 33 1983-1990,"Μάρκα αυτοκινήτου: Alfa Romeo, Μοντέλο: 33, Χρ...","9,03","12,19",1111,,test ΓΩΝΙΑ ΦΛΑΣ ΛΕΥΚΗ ΔΕ Alfa Romeo 33 1983-1990,ALFA ROMEO ALFA 33 83-90
5,Alfa Romeo,036107177-1,ΔΙΑΚΟΠΤΗΣ ΠΑΡΑΘΥΡΟ ΕΜΠΡΟΣ (Διπλός) (Καφέ φίσα)...,Γνήσιος κωδικός: 1Z0959,test ΔΙΑΚΟΠΤΗΣ ΠΑΡΑΘΥΡΟ ΕΜΠΡΟΣ (Διπλός) (Καφέ ...,"Μάρκα αυτοκινήτου: Alfa Romeo, Μοντέλο: 33, Χρ...","4,76","6,43",1111,https://ae01.alicdn.com/kf/H43be33263e8444cf92...,test ΔΙΑΚΟΠΤΗΣ ΠΑΡΑΘΥΡΟ ΕΜΠΡΟΣ (Διπλός) (Καφέ ...,"ALFA ROMEO ALFA 33 83-90, SEAT IBIZA 08-12, SE..."
6,Alfa Romeo,036107177-2,ΔΙΑΚΟΠΤΗΣ ΠΑΡΑΘΥΡΟ ΕΜΠΡΟΣ (Διπλός) (Καφέ φίσα)...,Γνήσιος κωδικός: 1Z0959858,test ΔΙΑΚΟΠΤΗΣ ΠΑΡΑΘΥΡΟ ΕΜΠΡΟΣ (Διπλός) (Καφέ ...,"Μάρκα αυτοκινήτου: Alfa Romeo, Μοντέλο: 33, Χρ...","4,76","6,43",1111,https://image.pushauction.com/0/0/ac95c77e-6ea...,test ΔΙΑΚΟΠΤΗΣ ΠΑΡΑΘΥΡΟ ΕΜΠΡΟΣ (Διπλός) (Καφέ ...,"ALFA ROMEO ALFA 33 83-90, SEAT IBIZA 08-12, SE..."
7,Alfa Romeo,065607482,ΚΑΘΡΕΦΤΗΣ ΜΗΧΑΝΙΚΟΣ ΜΕ ΝΤΙΖΕΣ ΑΡ Alfa Romeo 33...,Γνήσιος κωδικός:,test ΚΑΘΡΕΦΤΗΣ ΜΗΧΑΝΙΚΟΣ ΜΕ ΝΤΙΖΕΣ ΑΡ Alfa Rom...,"Μάρκα αυτοκινήτου: Alfa Romeo, Μοντέλο: 33, Χρ...","27,23","36,76",1111,,test ΚΑΘΡΕΦΤΗΣ ΜΗΧΑΝΙΚΟΣ ΜΕ ΝΤΙΖΕΣ ΑΡ Alfa Rom...,ALFA ROMEO ALFA 33 83-90
8,Alfa Romeo,065607481,ΚΑΘΡΕΦΤΗΣ ΜΗΧΑΝΙΚΟΣ ΜΕ ΝΤΙΖΕΣ ΔΕ Alfa Romeo 33...,Γνήσιος κωδικός:,test ΚΑΘΡΕΦΤΗΣ ΜΗΧΑΝΙΚΟΣ ΜΕ ΝΤΙΖΕΣ ΔΕ Alfa Rom...,"Μάρκα αυτοκινήτου: Alfa Romeo, Μοντέλο: 33, Χρ...","27,23","36,76",1111,,test ΚΑΘΡΕΦΤΗΣ ΜΗΧΑΝΙΚΟΣ ΜΕ ΝΤΙΖΕΣ ΔΕ Alfa Rom...,ALFA ROMEO ALFA 33 83-90
9,Alfa Romeo,065600070,ΚΑΠΩ ΕΜΠΡΟΣ Alfa Romeo 33 1983-1990,Γνήσιος κωδικός: 0060502461,test ΚΑΠΩ ΕΜΠΡΟΣ Alfa Romeo 33 1983-1990,"Μάρκα αυτοκινήτου: Alfa Romeo, Μοντέλο: 33, Χρ...","97,61","131,77",1111,https://static.car.gr/10294274_1_z.jpg,test ΚΑΠΩ ΕΜΠΡΟΣ Alfa Romeo 33 1983-1990,ALFA ROMEO ALFA 33 83-90
